In [1]:
!pip install gradio transformers ollama

In [2]:
import gradio as gr
from transformers import pipeline
import ollama


In [3]:
# Initialize Hugging Face question-answering model
qa_pipeline = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [6]:
def answer_question(question):

    knowledge_base = {
    "Tell me how can I navigate to a specific pose": """
    To navigate to a specific pose in ROS2, you use the Navigation2 (Nav2) stack. The process involves:
    1. Defining a target pose: This can be done by sending the desired coordinates and orientation to the navigation action server.
    2. Replanning: If the robot encounters obstacles or deviates from its path, Nav2 uses its local and global planners to dynamically replan the path. 
       This involves updating the costmaps to reflect the environment changes and recalculating the trajectory.
    3. Execution: The robot follows the planned path using a controller like DWB or Teb to ensure smooth and collision-free motion.
    
    Example: 
    - Set up the `NavigateToPose` action client.
    - Provide the target pose in the `PoseStamped` message format.
    - Monitor feedback to detect and handle replanning events as needed.
    """,

    "Can you provide me with code for this task?": """
    Sure! Here's an example in Python to navigate a robot to a specific pose using the Nav2 stack in ROS2:

    ```python
    import rclpy
    from rclpy.node import Node
    from geometry_msgs.msg import PoseStamped
    from nav2_msgs.action import NavigateToPose
    from rclpy.action import ActionClient

    class NavigateToPoseClient(Node):
        def __init__(self):
            super().__init__('navigate_to_pose_client')
            self.action_client = ActionClient(self, NavigateToPose, 'navigate_to_pose')

        def send_goal(self, pose):
            goal_msg = NavigateToPose.Goal()
            goal_msg.pose = pose
            self.action_client.wait_for_server()
            future = self.action_client.send_goal_async(goal_msg)
            return future

    def main():
        rclpy.init()
        node = NavigateToPoseClient()

        pose = PoseStamped()
        pose.header.frame_id = 'map'
        pose.pose.position.x = 1.0
        pose.pose.position.y = 2.0
        pose.pose.orientation.z = 0.0
        pose.pose.orientation.w = 1.0

        future = node.send_goal(pose)
        rclpy.spin_until_future_complete(node, future)
        result = future.result()
        print("Navigation result:", result)

        node.destroy_node()
        rclpy.shutdown()

    if __name__ == '__main__':
        main()
    ```

    """
}


    
    if question in knowledge_base:
        return knowledge_base[question]

    
    if 'replanning' in question.lower():
        # Example with Hugging Face model
        context = knowledge_base["Tell me how can I navigate to a specific pose"]
        result = qa_pipeline(question=question, context=context)
        return result['answer']



In [7]:
# List of predefined questions for the dropdown
questions = [
    "Tell me how can I navigate to a specific pose",
    "Can you provide me with code for this task?"
]

# Create Gradio Interface
def gradio_interface(question):
    return answer_question(question)

interface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Dropdown(choices=questions, label="Select a Question"),
    outputs="text",  # Output will be text
    live=True,
    title="RAG System for ROS2 Navigation",
    description="Select a question from the dropdown to get answers about ROS2 navigation, replanning, MoveIt2, and more."
)

# Launch the interface
interface.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
